In [2]:
import os
import xarray as xr
import geopandas as gpd
import rioxarray  # important: enables .rio

# ========= 1. PATHS =========
input_folder = r"Data\raster\addiation_process_1990\aus\qld_rainfall"            # your input .nc folder
output_folder = r"Data\raster\addiation_process_1990\ea\qld_rainfall"   # output folder
shapefile_path = r"Eastern_Aus_Bunbury\Eastern_Aus_Bunbury.shp"  # <-- update

os.makedirs(output_folder, exist_ok=True)

# ========= 2. READ SHAPEFILE =========
gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs("EPSG:4326")
shapes = gdf.geometry.values

# ========= 3. LOOP THROUGH FILES =========
for fname in os.listdir(input_folder):
    if not fname.endswith(".nc"):
        continue

    in_path = os.path.join(input_folder, fname)
    base = fname[:-3]  # strip ".nc"
    out_path = os.path.join(output_folder, f"{base}_ea.nc")

    print(f"Processing: {fname}")

    # Open dataset
    ds = xr.open_dataset(in_path)

    # ---- Tell rioxarray which dims are spatial ----
    # ABARES grid uses dims 'lon' and 'lat'
    # If your file uses something slightly different, print(ds.dims) to check.
    ds = ds.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=False)

    # Ensure CRS is set
    if ds.rio.crs is None:
        ds = ds.rio.write_crs("EPSG:4326")

    # ---- Clip ----
    try:
        ds_clipped = ds.rio.clip(
            shapes,
            ds.rio.crs,
            drop=True,
            from_disk=True
        )
    except Exception as e:
        print(f"  ⚠ Error clipping {fname}: {e}")
        ds.close()
        continue

    # Save clipped dataset
    ds_clipped.to_netcdf(out_path)

    ds.close()
    ds_clipped.close()

    print(f"  ✔ Saved: {out_path}")

print("All files clipped and saved with _cropped suffix.")


Processing: 1990.monthly_rain.nc
  ✔ Saved: Data\raster\addiation_process_1990\ea\qld_rainfall\1990.monthly_rain_ea.nc
All files clipped and saved with _cropped suffix.
